In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('formula1-originalDB.db')

query = '''
SELECT
    constructorId,
    name,
    nationality
FROM
    constructors;
'''
constructors_df = pd.read_sql_query(query, conn)

print(constructors_df)


     constructorId            name nationality
0                1         McLaren     British
1                2      BMW Sauber      German
2                3        Williams     British
3                4         Renault      French
4                5      Toro Rosso     Italian
..             ...             ...         ...
206            209  Manor Marussia     British
207            210    Haas F1 Team    American
208            211    Racing Point     British
209            213      AlphaTauri     Italian
210            214  Alpine F1 Team      French

[211 rows x 3 columns]


In [2]:
import pandas as pd
import sqlite3

# Assuming 'conn' is your SQLite connection
conn = sqlite3.connect('formula1.db')

# Your SQL query for constructor standings
constructor_points_query = '''
    WITH LastRacePerSeason AS (
    SELECT
        r.year,
        MAX(r.date) AS last_race_date
    FROM
        races r
    WHERE
        r.year BETWEEN 1950 AND 2022
    GROUP BY
        r.year
)

SELECT
    lrs.year,
    cs.constructorId,
    c.name AS constructor_name,
    cs.points,
    RANK() OVER (PARTITION BY lrs.year ORDER BY cs.points DESC) AS place
FROM
    LastRacePerSeason lrs
JOIN
    races r ON lrs.year = r.year AND lrs.last_race_date = r.date
JOIN
    constructor_standings cs ON r.raceId = cs.raceId
JOIN
    constructors c ON cs.constructorId = c.constructorId
ORDER BY
    lrs.year DESC, cs.points DESC;

'''

# Execute the query and create a DataFrame
constructor_points_df = pd.read_sql_query(constructor_points_query, conn)

# Display the resulting DataFrame
print(constructor_points_df)


     year  constructorId constructor_name  points  place
0    2022              9         Red Bull   759.0      1
1    2022              6          Ferrari   554.0      2
2    2022            131         Mercedes   515.0      3
3    2022            214   Alpine F1 Team   173.0      4
4    2022              1          McLaren   159.0      5
..    ...            ...              ...     ...    ...
895  1958            105         Maserati     6.0      5
896  1958             32       Team Lotus     3.0      6
897  1958            125        Connaught     0.0      7
898  1958            127             OSCA     0.0      7
899  1958             95          Porsche     0.0      7

[900 rows x 5 columns]


In [5]:
reshaped_query = '''
SELECT
    constructorId,
    JSON_GROUP_ARRAY(
        JSON_OBJECT(
            'year', year,
            'points', points,
            'place', place
        )
    ) AS years_points_places
FROM
    (
         WITH LastRacePerSeason AS (
    SELECT
        r.year,
        MAX(r.date) AS last_race_date
    FROM
        races r
    WHERE
        r.year BETWEEN 1950 AND 2022
    GROUP BY
        r.year
)

SELECT
    lrs.year,
    cs.constructorId,
    c.name AS constructor_name,
    cs.points,
    RANK() OVER (PARTITION BY lrs.year ORDER BY cs.points DESC) AS place
FROM
    LastRacePerSeason lrs
JOIN
    races r ON lrs.year = r.year AND lrs.last_race_date = r.date
JOIN
    constructor_standings cs ON r.raceId = cs.raceId
JOIN
    constructors c ON cs.constructorId = c.constructorId
ORDER BY
    lrs.year DESC, cs.points DESC;

'''

reshaped_df = pd.read_sql_query(reshaped_query, conn)

print(reshaped_df)


DatabaseError: Execution failed on sql '
SELECT
    constructorId,
    JSON_GROUP_ARRAY(
        JSON_OBJECT(
            'year', year,
            'points', points,
            'place', place
        )
    ) AS years_points_places
FROM
    (
         WITH LastRacePerSeason AS (
    SELECT
        r.year,
        MAX(r.date) AS last_race_date
    FROM
        races r
    WHERE
        r.year BETWEEN 1950 AND 2022
    GROUP BY
        r.year
)

SELECT
    lrs.year,
    cs.constructorId,
    c.name AS constructor_name,
    cs.points,
    RANK() OVER (PARTITION BY lrs.year ORDER BY cs.points DESC) AS place
FROM
    LastRacePerSeason lrs
JOIN
    races r ON lrs.year = r.year AND lrs.last_race_date = r.date
JOIN
    constructor_standings cs ON r.raceId = cs.raceId
JOIN
    constructors c ON cs.constructorId = c.constructorId
ORDER BY
    lrs.year DESC, cs.points DESC;

': near ";": syntax error